In [1]:
from robotic import ry
import numpy as np
import time

###########################################PART-2 STARTS HERE##########################################

In [2]:
C = ry.Config()
C.addFile('cargobot.g')
C.addFile('maze.g')
C.addFile('cargo.g')

qHome = C.getJointState()
C.setJointState(qHome)

Get Final Pose with KOMO

In [3]:
S = ry.Skeleton()

S.addEntry([1., 1.], ry.SY.touch,  ["l_gripper", "cargo_handle"])
#S.addEntry([1., 2.], ry.SY.stable,  ["l_panda_finger_joint1", "cargo_handle"])

#S.addEntry([1., 7.], ry.SY.stable,  ["l_gripper", "l_panda_finger_joint2"])
S.addEntry([1., 7.], ry.SY.stable,  ["l_gripper", "cargo_handle"])
#S.addEntry([0.8, 1.2], ry.SY.downUp, ["l_panda_finger_joint1"])
#S.addEntry([1.8, 2.2], ry.SY.downUp, ["l_panda_finger_joint1"])

S.addEntry([2., 2.], ry.SY.poseEq,  ["cargo", "cp1"])
#S.addEntry([2., -1.], ry.SY.stable,  ["cp1", "cargo"])

S.addEntry([3., 3.], ry.SY.poseEq,  ["cargo", "cp2"])
#S.addEntry([3., -1.], ry.SY.stable,  ["cp2", "cargo"])

S.addEntry([4., 4.], ry.SY.poseEq,  ["cargo", "cp3"])
#S.addEntry([4., -1.], ry.SY.stable,  ["cp3", "cargo"])

S.addEntry([5., 5.], ry.SY.poseEq,  ["cargo", "cp4"])
#S.addEntry([5., -1.], ry.SY.stable,  ["cp4", "cargo"])

S.addEntry([6., 6.], ry.SY.poseEq,  ["cargo", "cp5"])
#S.addEntry([6., -1.], ry.SY.stable,  ["cp5", "cargo"])

S.addEntry([7., 7.], ry.SY.poseEq,  ["cargo", "final"])
#S.addEntry([7., -1.], ry.SY.stable,  ["final", "cargo"])

S.enableAccumulatedCollisions(True)

komo = S.getKomo_waypoints(C, 1e-1, 1e-2,0)
nlp = komo.nlp()
sol = ry.NLP_Solver()
sol.setProblem(nlp)
sol.setOptions( stopTolerance=1e-2 )
ret = sol.solve()
waypoints = komo.getPath_qAll()
#waypoints = komo.getPath()[0]
komo.view_play(True, .2)
#C.setJointState(q[q.shape[0]-1])

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
==nlp== it:   0  evals:  30  A(x):  0.0161063  f:  0.0101904  g:          0  h:   0.124415  |x-x'|:    1.37598 	stop:DeltaConverge
==nlp== it:   1  evals:  30  A(x):  0.0516018  mu:5
==nlp== it:   1  evals:  32  A(x): 0.00965325  f:   0.010145  g:          0  h: 0.00834789  |x-x'|:  0.0154928 	stop:DeltaConverge
==nlp== it:   2  evals:  32  A(x):  0.0104126  mu:25
==nlp== it:   2  evals:  35  A(x):  0.0101871  f:  0.0101221  g:          0  h: 0.00304849  |x-x'|: 0.00125905 	stop:DeltaConverge
==nlp== StoppingCriterion Delta<0.01


1

RRT

In [4]:
#define a path finding problem
m = len(waypoints)
rrt_dofs = []
rrt_paths = []
for i in range(0, int(m)):

    [Ctmp, q0, q1] = S.getTwoWaypointProblem(i, komo)
    Ctmp.setJointState(q0)
    Ctmp.setJointState(q1)
    rrt = ry.PathFinder()
    rrt.setProblem(Ctmp, q0, q1)
    ret = rrt.solve()
    path = ret.x
    rrt_paths.append(rrt)
    rrt_dofs.append(Ctmp.getDofIDs())
    print("rettttt: ", ret)
    #print("path size:", path)
    # display the path
    """
    for i in range(0, path.shape[0]):
        Ctmp.setJointState(path[i])
        Ctmp.view(False)
        time.sleep(0.02)
        if hasattr(ret, 'x') and isinstance(ret.x, np.ndarray) and ret.x.ndim == 2 and ret.x.shape[0] > 0:
            rrt_paths.append(rrt)
            rrt_dofs.append(Ctmp.getDofIDs())
    """

rettttt:  { time: 0.00913, evals: 49, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }
rettttt:  { time: 2.29112, evals: 3194, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }
rettttt:  { time: 0.056359, evals: 87, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }
rettttt:  { time: 0.275843, evals: 572, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }
rettttt:  { time: 1.2998, evals: 2277, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }
rettttt:  { time: 0.921428, evals: 1351, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }
rettttt:  { time: 0.884352, evals: 1304, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }


In [5]:
#C.setJointState(qHome)
komo2 = S.getKomo_path(C, 60, 1e0, 1e0, 1e-2, 1e0)
komo2.initWithWaypoints(waypoints)


[59, 119, 179, 239, 299, 359, 419]

In [6]:

for t in range(0, int(m)):
    komo2.initPhaseWithDofsPath(t, rrt_dofs[t], rrt_paths[t], True)

ret = ry.NLP_Solver(komo2.nlp(), verbose=0).solve()
q = komo2.getPath()
print('last path:', q)

# display the path for part 1
for i in range(0, q.shape[0]):
    C.setJointState(q[i])
    C.view(False, f'waypoints{i}')
    time.sleep(0.02)

TypeError: initPhaseWithDofsPath(): incompatible function arguments. The following argument types are supported:
    1. (self: robotic.ry.KOMO, t_phase: int, dofIDs: Array<T>, path: arr, autoResamplePath: bool = False) -> None

Invoked with: <robotic.ry.KOMO object at 0x7f309cb6b730>, 0, [1, 6, 10, 14, 18, 22, 26, 30], <robotic.ry.PathFinder object at 0x7f309cb75db0>, True